In [49]:
# Import libraries
import pandas as pd
import datetime

In [50]:
# Load data with ferry stop times and location
df = pd.read_csv('data\google_transit\stop_times.txt')

df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
0,1,06:09:00,06:09:00,87,1,NaN,0,0,1
1,1,06:17:00,06:17:00,20,2,NaN,0,0,1
2,1,06:29:00,06:29:00,8,3,NaN,0,0,1
3,1,06:37:00,06:37:00,19,4,NaN,0,0,1
4,1,06:49:00,06:49:00,17,5,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...
3994,2262,05:04:00,05:04:00,52,5,NaN,0,0,1
3995,2262,05:06:00,05:06:00,53,6,NaN,0,0,1
3996,2262,05:12:00,05:12:00,16,7,NaN,0,0,1
3997,2288,05:16:00,05:16:00,16,1,NaN,0,0,1


In [51]:
# Create data frame with all possible times slots and respective time id
start_time = '4:45:00' # start time
end_time = '23:00:00' # end time
time_str_lst = pd.date_range(start_time, end_time, freq = '5min') # generate list with 5 min interval
time_lst = time_str_lst.strftime('%H:%M:%S') # modifty list to date.time format

df_times_reference = pd.DataFrame(data = time_lst, columns = ['time']) # create dataframe
df_times_reference['time_id'] = range(0, len(df_times_reference)) # add time_id reference

df_times_reference


,time,time_id
0,04:45:00,0
1,04:50:00,1
2,04:55:00,2
3,05:00:00,3
4,05:05:00,4
...,...,...
215,22:40:00,215
216,22:45:00,216
217,22:50:00,217
218,22:55:00,218


In [57]:
# Change name of time variable
df = df.rename(columns = {'arrival_time' : 'time'})

# Change "time" parameter from string to datetime format
try:
    df['time'] = pd.to_datetime(df['time']).dt.time
except Exception:
    pass

# Round time data to closest 5 min
df['time']= df['time']Timestamp..round(freq = '5T')

# Subset to relevent data
df_time = df[['trip_id','time']]

# Subset stop_id data
df_stop_id = df[['trip_id','stop_id']]

TypeError: round() got an unexpected keyword argument 'freq'

In [53]:
df_time = pd.merge(df, df_times_reference, on = 'time', how = 'left')

df_time

,trip_id,time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,time_id
0,1,06:09:00,06:09:00,87,1,NaN,0,0,1,NaN
1,1,06:17:00,06:17:00,20,2,NaN,0,0,1,NaN
2,1,06:29:00,06:29:00,8,3,NaN,0,0,1,NaN
3,1,06:37:00,06:37:00,19,4,NaN,0,0,1,NaN
4,1,06:49:00,06:49:00,17,5,NaN,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...
3994,2262,05:04:00,05:04:00,52,5,NaN,0,0,1,NaN
3995,2262,05:06:00,05:06:00,53,6,NaN,0,0,1,NaN
3996,2262,05:12:00,05:12:00,16,7,NaN,0,0,1,NaN
3997,2288,05:16:00,05:16:00,16,1,NaN,0,0,1,NaN


In [46]:
df_time = df_time.groupby('trip_id')['time'].apply(list).reset_index(name='time')

df_stop_id = df_stop_id.groupby('trip_id')['stop_id'].apply(list).reset_index(name='stop_id')

df = pd.merge(df_time,df_stop_id, on = 'trip_id', how = 'inner')

,trip_id,time,stop_id
0,1,"[06:09:00, 06:17:00, 06:29:00, 06:37:00, 06:49...","[87, 20, 8, 19, 17, 4]"
1,2,"[06:33:00, 06:41:00, 06:53:00, 07:01:00, 07:13...","[87, 20, 8, 19, 17, 4]"
2,4,"[06:57:00, 07:05:00, 07:17:00, 07:25:00, 07:37...","[87, 20, 8, 19, 17, 4]"
3,5,"[07:21:00, 07:29:00, 07:41:00, 07:49:00, 08:01...","[87, 20, 8, 19, 17, 4]"
4,6,"[07:45:00, 07:53:00, 08:05:00, 08:13:00, 08:25...","[87, 20, 8, 19, 17, 4]"
...,...,...,...
699,1903,"[21:05:00, 21:26:00, 21:38:00]","[137, 136, 138]"
700,1904,"[21:50:00, 22:11:00, 22:23:00]","[137, 136, 138]"
701,2239,"[05:16:00, 05:44:00]","[16, 62]"
702,2262,"[04:57:00, 04:59:00, 05:00:00, 05:02:00, 05:04...","[48, 49, 50, 51, 52, 53, 16]"


In [ ]:
export_lst = ['time', 'trip_id', 'latitude', 'longitude','time_id']

df_export = df_merge[export_lst]


df_export.to_csv(r'stop-times.csv', index = False, header=True)